In [1]:
import pandas as pd 
from tabulate import tabulate
import numpy as np

#### Config

In [2]:
Config = {
    "products": ["domain", "category", "subcategory", "further_subcategory", "brand", "list_price", "sale_price", "sales_volume", 'best_sellers_rank', 'accessories', 'url', 'image_url_1', 'source'],
    "products_extend": ['source_product_id', 'extend_class',
       'extend_subclass', 'extend_detail_raw', 'extend_detail', 'extend_unit', 'source',
       'domain', 'category', 'subcategory', 'further_subcategory',
       'original_extend_detail',
       'original_extend_unit']
}
classification_columns = ["domain", "category", "subcategory", "further_subcategory"]


In [3]:
classification = pd.read_excel("source/設研院產品分類表.xlsx")[["domain", "category", "subcategory", "further_subcategory"]]
classification['classification'] = classification.apply(lambda row: "_".join([row[col] for col in classification.columns]), axis = 1)

#### Products

In [3]:
products = pd.read_excel("../inputs_0715/products_momo_0715_delete.xlsx")

In [58]:
for col in Config['products']:
    if col not in products.columns:
        print(f"⚠️ missing column: {col}")

In [111]:
table = (pd.DataFrame(products[Config['products']].isna().sum())
         .rename(columns = {0: "count"}))
table['proportion'] = table['count'] / len(products)
table = table.astype("object")
table['count'] = table['count'].apply(lambda x: f"{int(x)}")
table['proportion'] = table['proportion'].apply(lambda x: f"{round(x, 3) * 100} %")
table

,count,proportion
domain,2852,5.5 %
category,2852,5.5 %
subcategory,2852,5.5 %
further_subcategory,2852,5.5 %
brand,0,0.0 %
list_price,4040,7.8 %
sale_price,4040,7.8 %
sales_volume,31861,61.5 %
best_sellers_rank,51826,100.0 %
accessories,48892,94.3 %


In [112]:
print("Products - 重要欄位空值分析")
print(tabulate(table, headers='keys', tablefmt='fancy_grid', numalign = "right"))

Products - 重要欄位空值分析
╒═════════════════════╤═════════╤══════════════╕
│                     │   count │ proportion   │
╞═════════════════════╪═════════╪══════════════╡
│ domain              │    2852 │ 5.5 %        │
├─────────────────────┼─────────┼──────────────┤
│ category            │    2852 │ 5.5 %        │
├─────────────────────┼─────────┼──────────────┤
│ subcategory         │    2852 │ 5.5 %        │
├─────────────────────┼─────────┼──────────────┤
│ further_subcategory │    2852 │ 5.5 %        │
├─────────────────────┼─────────┼──────────────┤
│ brand               │       0 │ 0.0 %        │
├─────────────────────┼─────────┼──────────────┤
│ list_price          │    4040 │ 7.8 %        │
├─────────────────────┼─────────┼──────────────┤
│ sale_price          │    4040 │ 7.8 %        │
├─────────────────────┼─────────┼──────────────┤
│ sales_volume        │   31861 │ 61.5 %       │
├─────────────────────┼─────────┼──────────────┤
│ best_sellers_rank   │   51826 │ 100.0 %      │


In [115]:
for col in ["domain", "category", "subcategory", "further_subcategory"]:
    if int(table.loc[col, "count"]) > 0:
        print(f"⚠️ 有 {int(table.loc[col, "count"])} 列缺失類別欄位 {col}")

⚠️ 有 2852 列缺失類別欄位 domain
⚠️ 有 2852 列缺失類別欄位 category
⚠️ 有 2852 列缺失類別欄位 subcategory
⚠️ 有 2852 列缺失類別欄位 further_subcategory


In [85]:
print("正在檢查分類組合...")
incorrect_classified_ids = []
for _, row in products.dropna(subset = classification_columns).iterrows():
    class_ = "_".join([row[col] for col in classification_columns]) 
    if class_ not in classification['classification'].tolist():
        incorrect_classified_ids.append(row['id'])

print(f"⚠️ 共有 {len(incorrect_classified_ids)} 比資料的分類組合不存在於分類資料表中")

正在檢查分類組合...
⚠️ 共有 466 比資料的分類組合不存在於分類資料表中


#### Product Extend

In [20]:
products_extend = pd.read_csv("../inputs_0715/products_extend_momo_0715_delete.csv")

In [ ]:
def extend_subclass_null_analysis()

In [47]:
(products_extend
 .groupby("extend_class")
 .apply(lambda group: 
        pd.DataFrame(
                {
                "na_extend_subclass_count": pd.Series(group["extend_subclass"].isna().sum()),
                "proportion":  pd.Series(group["extend_subclass"].isna().sum() / len(group))
             }   
        )
        , include_groups=  False))

,,na_extend_subclass_count,proportion
extend_class,,,
使用情境,0,100161,1.000000
保固,0,27914,0.999427
功能,0,140750,1.000000
容量,0,0,0.000000
尺寸,0,0,0.000000
尺寸_收納尺寸,0,0,0.000000
尺寸_部件尺寸,0,0,0.000000
效能,0,0,0.000000
族群,0,0,0.000000


In [121]:
for col in Config['products_extend']:
    if col not in products_extend.columns:
        print(f"⚠️ missing column: {col}")

In [124]:
table = (pd.DataFrame(products_extend[Config['products_extend']].isna().sum())
         .rename(columns = {0: "count"}))
table['proportion'] = table['count'] / len(products_extend)
table = table.astype("object")
table['count'] = table['count'].apply(lambda x: f"{int(x)}")
table['proportion'] = table['proportion'].apply(lambda x: f"{round(x, 2) * 100} %")

print("Products - 重要欄位空值分析")
print(tabulate(table, headers='keys', tablefmt='fancy_grid', numalign = "right"))

,count,proportion
source_product_id,0,0.0 %
extend_class,4040,0.0 %
extend_subclass,579399,71.0 %
extend_detail_raw,173825,21.0 %
extend_detail,4041,0.0 %
extend_unit,645596,79.0 %
source,4040,0.0 %
domain,173825,21.0 %
category,4040,0.0 %
subcategory,4040,0.0 %
